In [15]:
import numpy as np
import scipy.signal
import torch
import torch.nn as nn

In [8]:
def convld(x, w, p=0, s=1):
    w_rot = np.array(w[::-1])

    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p)
        x_padded = np.concatenate([zero_pad, x_padded, zero_pad])

    res = []
    for i in range(0, (int((len(x_padded) - len(w_rot)) / s) + 1) * s, s):
        res.append(np.sum(x_padded[i: i + w_rot.shape[0]] * w_rot))

    return np.array(res)

In [9]:
x = [1, 3, 2, 4, 5, 6, 1, 3]
w = [1, 0, 3, 1, 2]
convld(x, w, p=2, s=1)

array([ 5., 14., 16., 26., 24., 34., 19., 22.])

In [5]:
np.convolve(x, w, mode='same')

array([ 5, 14, 16, 26, 24, 34, 19, 22])

In [11]:
def conv2d(X, W, p=(0, 0), s=(1, 1)):
    W_rot = np.array(W)[::-1, ::-1]
    X_orig = np.array(X)
    n1 = X_orig.shape[0] + 2 * p[0]
    n2 = X_orig.shape[1] + 2 * p[1]
    X_padded = np.zeros(shape=(n1, n2))
    X_padded[p[0]: p[0] + X_orig.shape[0], p[1]: p[1] + X_orig.shape[1]] = X_orig
    res = []
    for i in range(0, (int((X_padded.shape[0] - W_rot.shape[0]) / s[0]) + 1) * s[0], s[0]):
        res.append([])
        for j in range(0, (int((X_padded.shape[1] - W_rot.shape[1]) / s[1]) + 1) * s[1], s[1]):
            X_sub = X_padded[i: i + W_rot.shape[0], j: j + W_rot.shape[1]]
            res[-1].append(np.sum(X_sub * W_rot))
    return np.array(res)

In [12]:
X = [[1, 3, 2, 4], [5, 6, 1, 3], [1, 2, 0, 2], [3, 4, 3, 2]]
W = [[1, 0, 3], [1, 2, 1], [0, 1, 1]]
conv2d(X, W, p=(1, 1), s=(1, 1))

array([[11., 25., 32., 13.],
       [19., 25., 24., 13.],
       [13., 28., 25., 17.],
       [11., 17., 14.,  9.]])

In [13]:
scipy.signal.convolve2d(X, W, mode='same')

array([[11, 25, 32, 13],
       [19, 25, 24, 13],
       [13, 28, 25, 17],
       [11, 17, 14,  9]])

In [16]:
loss_func = nn.BCELoss()
loss = loss_func(torch.tensor([0.9]), torch.tensor([1.0]))
l2_lambda = 0.001
conv_layer = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=5)
l2_penalty = l2_lambda * sum([(p**2).sum() for p in conv_layer.parameters()])
loss_with_penalty = loss + l2_penalty
linear_layer = nn.Linear(10, 16)
l2_penalty = l2_lambda * sum([(p**2).sum() for p in linear_layer.parameters()])
loss_with_penalty = loss + l2_penalty

In [17]:
logits = torch.tensor([0.8])
probas = torch.sigmoid(logits)
target = torch.tensor([1.0])
bce_loss_fn = nn.BCELoss()
bce_logits_loss_fn = nn.BCEWithLogitsLoss()
bce_loss_fn(probas, target)

tensor(0.3711)

In [18]:
bce_logits_loss_fn(logits, target)

tensor(0.3711)

In [19]:
logits = torch.tensor([[1.5, 0.8, 2.1]])
probas = torch.softmax(logits, dim=1)
target = torch.tensor([2])
cce_loss_fn = nn.NLLLoss()
cce_logits_loss_fn = nn.CrossEntropyLoss()
cce_logits_loss_fn(logits, target)

tensor(0.5996)

In [21]:
cce_loss_fn(torch.log(probas), target)

tensor(0.5996)